## Semantic segmentation 

### This is a multi-class semantic segmentation problem. Let's solve it using YOLOv8

In [1]:
# Importing necessary libraries
import os
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import random
import copy
import shutil
import numpy as np
import ipywidgets as widgets
import wandb

In [2]:
# Installing ultralytics
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 984.0/984.0 kB 19.3 MB/s eta 0:00:00a 0:00:01


## Creating working directories

In [ ]:
# Defining input images path, output path, masks path and labels path
path='/kaggle/input/cholecseg8k'
op_path='/kaggle/working'
rawimages_path=os.path.join(op_path, 'raw_images')
maskimages_path=os.path.join(op_path, 'mask_images')
labels_path=os.path.join(op_path, 'labels')
os.makedirs(rawimages_path)
os.makedirs(maskimages_path)
os.makedirs(labels_path)

In [ ]:
# Defining output train,val and test paths
imgtrainpath = os.path.join(op_path,'images','train')
imgvalpath=os.path.join(op_path,'images','validation')
imgtestpath=os.path.join(op_path,'images','test')

labeltrainpath=os.path.join(op_path,'labels','train')
labelvalpath=os.path.join(op_path,'labels','validation')
labeltestpath=os.path.join(op_path,'labels','test')

os.makedirs(imgtrainpath)
os.makedirs(imgvalpath)
os.makedirs(imgtestpath)

os.makedirs(labeltrainpath)
os.makedirs(labelvalpath)
os.makedirs(labeltestpath)

In [ ]:
# Defining progress bar for visualising progress of long processes
progress_bar=widgets.FloatProgress(value=0, min=0, max=100, description='Progress', 
                                   layout=widgets.Layout(width='100%'))
progress_bar

Now we transfer raw images and their corresponding color masks to their relevant output directories. Also we calculate the total number of sub-directories, total images, raw images and mask images 

In [ ]:
m=0 # variable for counting total sub-directories
n=0 # variable for counting total images
o=0 # variable for counting total raw images
p=0 # variable for counting total masks
q=0 # variable for counting total directories
for directory in os.listdir(path):
    dir_path=os.path.join(path, directory)
    m=m+len(os.listdir(dir_path))
    q=q+1
    for sub_dir in os.listdir(dir_path):
        sub_dir_path=os.path.join(dir_path, sub_dir)
        n=n+len(os.listdir(sub_dir_path))
        for image in os.listdir(sub_dir_path):
            src_path=os.path.join(sub_dir_path, image)
            # Rename the image based on sub-directory to distinguish images with same names from different directories
            newname=sub_dir+image
            if 'mask' not in image:   # Criterion for raw image             
                newpath=os.path.join(rawimages_path, newname)
                dest_path=os.path.join(rawimages_path, image)
                shutil.copy(src_path, dest_path) # Copying raw image to output directory
                os.rename(dest_path, newpath) # Renaming raw image
                o=o+1
            if 'color_mask' in image:  # Criterion for color mask  
                newpath=os.path.join(maskimages_path, newname)
                dest_path=os.path.join(maskimages_path, image)
                shutil.copy(src_path, dest_path) # Copying mask to output directory
                os.rename(dest_path, newpath) # Renaming mask
                p=p+1
    # Updating progress bar to visualise copying and renaming of raw images and masks            
    progress_bar.value=q/len(os.listdir(path))*100 


In [ ]:
print("Total number of sub-directories:", m)
print("Total number of images:", n)
print("Total number of raw images:", o)
print("Total number of masks:", p)


We see here that we have equal number of raw images and masks. Let's verify if all the raw images and masks have been copied

In [ ]:
# checking if all raw images and masks have been copied successfully
len(os.listdir(rawimages_path)), len(os.listdir(maskimages_path))

All images and masks have been successfully copied

In [ ]:
# Checking first five raw images
os.listdir(rawimages_path)[:5]

In [ ]:
# Checking first five masks
os.listdir(maskimages_path)[:5]

We see that the raw images and masks are not sorted in order. So we need to arrange them in order to visualize them as image-mask pairs

In [ ]:
# Sorting raw images and masks
rawimages_list=sorted(os.listdir(rawimages_path))
maskimages_list=sorted(os.listdir(maskimages_path))

In [ ]:
# Checking first five sorted raw images
rawimages_list[:5]

In [ ]:
# Checking first five sorted masks
maskimages_list[:5]

So raw images and masks are now sorted.

## Visualising images and masks

I have run some tests to find out the all the different RGB colors present in colored masks corresponding to the class labels and the raw images given in the problem description. Here I define the color-class mapping based on the result of my tests

In [ ]:
color_class_mapping={(127, 127, 127): 0,
                    (210, 140, 140): 1,
                    (255, 114, 114): 2,
                    (231, 70, 156): 3,
                    (186, 183, 75): 4,
                    (170, 255, 0): 5,
                    (255, 85, 0): 6,
                    (255, 0, 0): 7,
                    (255, 255, 0): 8,
                    (169, 255, 184): 9,
                    (255, 160, 165): 10,
                    (0, 50, 128): 11,
                    (111, 74, 0): 12}



Similarly, class-color mapping is defined as well. It's the same mapping, just the other way around. It maps classes to their respective colors. It will be useful for post-processing the predicted image.

In [ ]:
class_color_mapping = {class_index: color for color, class_index in color_class_mapping.items()}

The class to name mapping as given in the problem description is as follows:

In [ ]:
class_name_mapping={0: 'Black Background',
                    1: 'Abdominal Wall',
                    2: 'Liver',
                    3: 'Gastrointestinal Tract',
                    4: 'Fat',
                    5: 'Grasper',
                    6: 'Connective Tissue',
                    7: 'Blood',
                    8: 'Cystic Duct',
                    9: 'L-hook Electrocautery',
                    10: 'Gallbladder',
                    11: 'Hepatic Vein',
                    12: 'Liver Ligament'}

Now I will write a function to display raw-image and its corresponding mask

In [ ]:
def plot_image_and_mask():
    figure,axis = plt.subplots(1,2,figsize=(30,30))
    plt.axis('off')
    k=random.randint(0, len(os.listdir(rawimages_path))-1) # choosing any random image number
    
    img_path=os.path.join(rawimages_path, rawimages_list[k]) # defining image path
    mask_path=os.path.join(maskimages_path, maskimages_list[k]) # defining mask path
    
    img_title=os.path.basename(img_path) # extracting image filename from path
    mask_title=os.path.basename(mask_path) # extracting mask filename from path
    
    # displaying image and mask
    axis[0].imshow(cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB))
    axis[0].set_title(img_title, fontsize=30)
    axis[0].set_xticks([])
    axis[0].set_yticks([])
    axis[1].imshow(cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2RGB))
    axis[1].set_title(mask_title, fontsize=30)
    axis[1].set_xticks([])
    axis[1].set_yticks([])

    plt.tight_layout()
    plt.show()

In [ ]:
plot_image_and_mask()

In [ ]:
plot_image_and_mask()

In [ ]:
plot_image_and_mask()

In [ ]:
plot_image_and_mask()

In [ ]:
plot_image_and_mask()

In [ ]:
plot_image_and_mask()

## Masks visualisation with contours

We will find unique colors in mask and draw segmentation contours for any color present in it. Since we don't have pure black color (RGB-0,0,0) for any of the classes in masks, we will use it to draw contours.

In [ ]:
# defining black color to draw contour on mask
black=(0,0,0)

In [ ]:
# function to draw contour around any random color present in the mask
def draw_contour_for_one_color_on_mask():
    figure,axis = plt.subplots(1,3,figsize=(30,30))
    plt.axis('off')
    k=random.randint(0, len(rawimages_list)-1) # choosing any random image
    
    img_path=os.path.join(rawimages_path, rawimages_list[k]) # defining image path
    img_title=os.path.basename(img_path) # extracting basename from path
    img=cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    
    mask_path=os.path.join(maskimages_path, maskimages_list[k]) # defining mask path
    mask_title='Mask' 
    mask=cv2.cvtColor(cv2.imread(mask_path), cv2.COLOR_BGR2RGB)
    
    mask_copy=copy.deepcopy(mask) # creating a copy of mask to draw contour
    pixels = mask.reshape((-1, 3))
    unique_colors = np.unique(pixels, axis=0) # getting unique colors present in mask
    
    #getting only those unique colors which are defined in problem
    unique_colors_defined = [value for value in unique_colors if tuple(value) in color_class_mapping]
    unique_colors_defined = np.array(unique_colors_defined, dtype=np.uint8)
    total_colors=len(unique_colors_defined)
    j=random.randint(0, total_colors-1) # selecting any random color among all the defined colors present in mask
    color=unique_colors_defined[j]
    # defining title for contour on mask
    mask_copy_title='Mask with contour on ' +str(class_name_mapping[color_class_mapping[tuple(color)]])
    
    mask_mask = cv2.inRange(mask, color, color) # getting mask of the selcted color on the mask-image
    contours, hrc = cv2.findContours(mask_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # finding contours
    cv2.drawContours(mask_copy, contours, -1, black, 4) # drawing contours
    
    axis[0].imshow(img) # displaying image
    axis[0].set_title(img_title, fontsize=30)
    axis[0].set_xticks([])
    axis[0].set_yticks([])
    axis[1].imshow(mask) # displaying mask 
    axis[1].set_title(mask_title, fontsize=30)
    axis[1].set_xticks([])
    axis[1].set_yticks([])
    axis[2].imshow(mask_copy) # displaying copy of mask with contour 
    axis[2].set_title(mask_copy_title, fontsize=30)
    axis[2].set_xticks([])
    axis[2].set_yticks([])

    plt.tight_layout()
    plt.show()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

In [ ]:
draw_contour_for_one_color_on_mask()

## Writing text file for masks

Now that there is a way to extract the contours, we will define a function to write text file per mask for all the contours for all the colors present in the mask. 

In [ ]:
def write_polygon_file(class_contour_mapping, H, W, output_path, img_name):
    coordinates={}
    for obj in class_contour_mapping: # looping through all colors present in the mask
        polygons = []
        for cnt in class_contour_mapping[obj]: # looping through all contours present in the color
            if cv2.contourArea(cnt) > 20: # neglecting very small contours
                polygon = []
                for point in cnt: # looping through all points present in the contour
                    x, y = point[0]
                    polygon.append(round(x / W, 4))
                    polygon.append(round(y / H, 4))
                polygons.append(polygon)
        coordinates[obj]=polygons

    # creating text file for all contours of all colors present in mask
    with open('{}.txt'.format(os.path.join(output_path, img_name)), 'w') as f:
        for obj in coordinates:
            for polygon in coordinates[obj]:
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:  # if point is the last point in contour, need to give newline
                        f.write('{}\n'.format(p))
                    elif p_ == 0: # if point is the first point in contour, need to specify color also
                        f.write('{} {} '.format(obj, p))
                    else: # any other point between first and last
                        f.write('{} '.format(p))

We will now create segmentation label text files for all the masks. Let's use the progress bar again to visually check the progress of writing text files.

In [ ]:
# Restarting the progress bar to track the progress of test label files creation
progress_bar.value=0
progress_bar

In [ ]:
k=0 # for counting total number of labels created

for img in maskimages_list:
    # extracting shortened mask name (i.e mask name upto 'endo')
    parts=img.split('_')
    endo_index=parts.index('endo')
    newname='_'.join(parts[:endo_index+1])
    
    # reading the image
    image=cv2.cvtColor(cv2.imread(os.path.join(maskimages_path,img)), cv2.COLOR_BGR2RGB)
    
    # getting unique colors present in mask
    pixels = image.reshape((-1, 3))
    unique_colors = np.unique(pixels, axis=0) 
    
    #getting only those unique colors which are defined in problem
    unique_colors_defined = [value for value in unique_colors if tuple(value) in color_class_mapping]
    unique_colors_defined = np.array(unique_colors_defined, dtype=np.uint8)
    total_colors=len(unique_colors_defined)

    H,W,_=image.shape # extracting mask dimensions
    class_contour_mapping={}

    for i in range(total_colors): # looping through all colors present in mask
        color=unique_colors_defined[i] # extracting the color
        class_code=color_class_mapping[tuple(color)] # getting color-code
        mask = cv2.inRange(image, color, color) # getting mask of color on the mask-image
        contours, hrc = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) # finding contours
        class_contour_mapping[class_code]=contours # mapping color-code to contours
            
    # writing label text file    
    write_polygon_file(class_contour_mapping, H, W, labels_path, newname)
    k=k+1
    
    progress_bar.value=k/len(maskimages_list)*100    # updating progress bar
    
print("Total number of labels created: ", k)    

## Training, Validation and Test dataset

We will now shuffle the dataset randomly and then create training, validation and test dataset from the raw images. Let's write a function to do it.

In [ ]:
# function to create training, validation and test dataset
def create_dataset(images_list):
    random.shuffle(images_list)
    train_images=images_list[:int(0.8*len(images_list))]
    val_images=images_list[int(0.8*len(images_list)):int(0.9*len(images_list))]
    test_images=images_list[int(0.9*len(images_list)):]
    return train_images, val_images, test_images

In [ ]:
train_images, val_images, test_images=create_dataset(rawimages_list)

In [ ]:
# checking size of training, validation and test dataset
len(train_images), len(val_images), len(test_images)

Now that the dataset is bifurcated, we will write a function to create names of label files corresponding to the names of image files

In [ ]:
# defining the extension for the text label files
extension='.txt'

In [ ]:
# function to extract basename from a file and add a different extension to it. 
def change_extension(file):
    basename=os.path.splitext(file)[0]
    filename=basename+extension
    return filename

In [ ]:
# creating a list of label files corresponding to the image files for each dataset 
train_labels = list(map(change_extension, train_images)) 
val_labels = list(map(change_extension, val_images)) 
test_labels = list(map(change_extension, test_images)) 

Let's verify the size of labels in datasets

In [ ]:
len(train_labels), len(val_labels), len(test_labels)

Now that size of labels is same as that of images in training, validation and test sets, let's write functions to move images and labels to their respective directories

While moving the images, we also resize them to a fixed size so that all images have exactly same size.

In [ ]:
# defining new image size for all images
image_size=640

In [ ]:
# function to resize the images and copy the resized image to destination
def move_images(data_list, source_path, destination_path):
    i=0
    for file in data_list:
        filepath=os.path.join(source_path, file)
        finalimage_path=os.path.join(destination_path, file)
        img_resized=cv2.resize(cv2.imread(filepath), (image_size, image_size))
        cv2.imwrite(finalimage_path, img_resized)
        i=i+1
    print("Number of files transferred:", i)

In [ ]:
# function to move files present in a list from source to destination
def move_files(data_list, source_path, destination_path):
    i=0
    for file in data_list:
        filepath=os.path.join(source_path, file)
        shutil.move(filepath, destination_path)
        i=i+1
    print("Number of files transferred:", i)

In [ ]:
# moving training images
move_images(train_images, rawimages_path, imgtrainpath)

In [ ]:
# moving validation images
move_images(val_images, rawimages_path, imgvalpath)

In [ ]:
# moving test images
move_images(test_images, rawimages_path, imgtestpath)

In [ ]:
# moving train labels
move_files(train_labels, labels_path, labeltrainpath)

In [ ]:
# moving validation labels
move_files(val_labels, labels_path, labelvalpath)

In [ ]:
# moving test labels
move_files(test_labels, labels_path, labeltestpath)

So, all the images and labels have been moved to their respective directories. 

Let's have the list of test masks handy as it will be required later for visualisation 

In [ ]:
# creating list of test masks for visualisation 
test_masks=list(map(lambda x:os.path.splitext(x)[0]+'_color_mask.png', test_images))

Now we are good to create configuration file.

## Creating config file

The config file is required to use YOLOv8 model. The names of classes present in dataset and the directories for the training, validation and test datasets are indicated in the config file.

In [ ]:
# defining newline variable for config file
newline='\n'

Let's define the contents of the config file.

In [ ]:
# defining lines of config text file

ln_1='# Train/val/test sets'+newline # starting with a comment line

# train, val and test path declaration
ln_2='train: ' +"'"+imgtrainpath+"'"+newline
ln_3='val: ' +"'" + imgvalpath+"'"+newline
ln_4='test: ' +"'" + imgtestpath+"'"+newline
ln_5=newline

# names of the classes declaration
ln_6='# Classes'+newline
ln_7='names:'+newline
ln_8='  0: Black Background'+newline
ln_9='  1: Abdominal Wall'+newline
ln_10='  2: Liver'+newline
ln_11='  3: Gastrointestinal Tract'+newline
ln_12='  4: Fat'+newline
ln_13='  5: Grasper'+newline
ln_14='  6: Connective Tissue'+newline
ln_15='  7: Blood'+newline
ln_16='  8: Cystic Duct'+newline
ln_17='  9: L-hook Electrocautery'+newline
ln_18='  10: Gallbladder'+newline
ln_19='  11: Hepatic Vein'+newline
ln_20='  12: Liver Ligament'

#listing all config lines
config_lines=[ln_1, ln_2, ln_3, ln_4, ln_5, ln_6, ln_7, ln_8, ln_9, ln_10, ln_11, ln_12,
             ln_13, ln_14, ln_15, ln_16, ln_17, ln_18, ln_19, ln_20]

In [ ]:
# Creating path for config file
config_path=os.path.join(op_path, 'config.yaml')
config_path

Now, we create the config file.

In [ ]:
# Writing config file
with open(config_path, 'w') as f:
    f.writelines(config_lines)

## Model training

Now we can start the training process using YOLO's pretrained model

In [ ]:
# Using YOLO's pretrained model architecture and weights for training
model=YOLO('yolov8m-seg.yaml').load('yolov8m-seg.pt')

In [ ]:
# Training the model for 30 epochs; here degrees, shear and perspective are augmentation arguments
results=model.train(data=config_path, epochs=30, iou=0.4, conf=0.01, degrees=60, 
                    shear=30, perspective=0.0005)

Results can be converted to a zip file using the following command which is commented right now. This zip file can be downloaded later if results are to be analysed locally

In [ ]:
# !zip -r results.zip /kaggle/working/runs/segment/train

Let's see how the training progressed with epochs by visualizing the plots

In [ ]:
plt.figure(figsize=(30,30))
trainingresult_path=os.path.join(op_path, 'runs', 'segment', 'train')
results_png=cv2.imread(os.path.join(trainingresult_path,'results.png'))
plt.imshow(results_png)

All losses- Box loss, seg loss, class loss, dfl loss are decreasing with epochs. All metrics- Precision, Recall, mAP50 and mAP50-95 are increasing with epochs

## Model performance and visualisation

Now, we will write a function to evaluate mAP50. mAP50 is mean Average Precision at IoU=0.5. It is a metric measuring average precision of segmentation masks at IoU(Intersection over Union)=0.5

In [ ]:
# function for evaluating model metrics map50
def evaluate_map50(trainedmodel, data_path, dataset='val'):
    metrics=trainedmodel.val(data=data_path, split=dataset)
    map50=round(metrics.seg.map50, 3)
    print("The mAP of model for all images on {0} dataset is {1}".format(dataset,map50))
    return metrics, map50

Let's evaluate mAP50 on test dataset

In [ ]:
# Evaluating test metrics
test_metrics, test_map50=evaluate_map50(model, config_path, dataset='test')

Now we should visualise the performance of our model using some test images

Let's first write a function to postprocess the predicted image from the model. This is done to visualise the predicted image in custom colors.

In [ ]:
# function to post-process predicted image with custom colors
def postprocess_prediction(prediction):
    masks=[]
    
    #collecting all masks after coloring them
    for i in range(len(prediction.boxes.cls)):
        background = np.zeros_like(prediction.orig_img) # creating a black background of same size as image
        color=class_color_mapping[int(prediction.boxes.cls[i])] # extracting custom color based on predicted class from class-color mapping 
        mask_points=prediction.masks[i].xy[0].astype(np.int32) # obtaining contours of mask from predicted image
        cv2.fillPoly(background, [mask_points], color); # filling background with mask contour using extracted color
        masks.append(background)
    
    #joining all masks such that no mask is superimposed on the previous mask but only black pixels are modified
    for i in range(len(masks)-1):
        zero_color_mask = np.all(masks[0] == [0, 0, 0], axis=2) # creating mask of black pixels in first mask
        masks[0][zero_color_mask] = masks[i+1][zero_color_mask] # coloring black pixels of first mask with color of subsequent mask
        
    return masks[0]

In [ ]:
# function to visualise model performance on test images
def visualise_model_performance():
    plt.figure(figsize=(30,30))
    plt.axis('off')
    k=random.randint(0, len(test_images)-1) # choosing any random image
    
    test_image=os.path.join(rawimages_path, test_images[k]) # defining path of image
    img_title=os.path.basename(test_image) # extracting basename from image path
    img=cv2.cvtColor(cv2.imread(test_image), cv2.COLOR_BGR2RGB) # reading image

    
    test_mask=os.path.join(maskimages_path, test_masks[k]) # defining path of mask
    mask_title=os.path.basename(test_mask) # extracting basename from mask path
    mask=cv2.cvtColor(cv2.imread(test_mask), cv2.COLOR_BGR2RGB) # reading mask
    
    pred = model(test_image) # predicting on image
    pred_plotted = pred[0].plot(boxes=False) # prediction without bounding box displayed
    pred_plotted=cv2.cvtColor(pred_plotted, cv2.COLOR_BGR2RGB)
    pred_title='Model Prediction' # title for prediction
    
    postprocessed_image=postprocess_prediction(pred[0]) # post-processing prediction
    postprocessed_image_title='Post-processed prediction' # title for post-processed prediction
    
    ax=plt.subplot(2,2,1)
    plt.imshow(img)# displaying image
    plt.title(img_title, fontsize=30)
    plt.xticks([])
    plt.yticks([])
    ax=plt.subplot(2,2,2)
    plt.imshow(mask)# displaying mask
    plt.title(mask_title, fontsize=30)
    plt.xticks([])
    plt.yticks([])
    
    ax=plt.subplot(2,2,3)
    plt.imshow(pred_plotted)# displaying prediction
    plt.title(pred_title, fontsize=30)
    plt.xticks([])
    plt.yticks([])
    ax=plt.subplot(2,2,4)
    plt.imshow(postprocessed_image)# displaying post-processed prediction
    plt.title(postprocessed_image_title, fontsize=30)
    plt.xticks([])
    plt.yticks([])

    plt.tight_layout()
    plt.show()
    

Now, let's see the predictions

In [ ]:
visualise_model_performance()

In [ ]:
visualise_model_performance()

In [ ]:
visualise_model_performance()

In [ ]:
visualise_model_performance()

In [ ]:
visualise_model_performance()

In [ ]:
visualise_model_performance()

In [ ]:
visualise_model_performance()

In [ ]:
visualise_model_performance()

We see here the performance of YOLOv8 model on semantic segmentation. The post-processed predicted images are extremely accurate. This can be vital for computer-assisted surgery. Thanks for checking.